### Predicting water level for the next 2 weeks to a month and writing the predictions out

In [217]:
import argparse
import pandas as pd
import hopsworks
import os

from xgboost import XGBRegressor

parser = argparse.ArgumentParser()
parser.add_argument("--latitude", type=float, default=59.3284)
parser.add_argument("--longitude", type=float, default=18.0664)
parser.add_argument("--sensor-id", type=int, default=20389)
parser.add_argument("--sensor-name", type=str, default="malaren_w")
parser.add_argument("--water-level-fg-version", type=int, default=4)
parser.add_argument("--weather-fg-version", type=int, default=4)
parser.add_argument("--fv-version", type=int, default=2)
parser.add_argument("--model-version", type=int, default=1)
parser.add_argument("--sensor-url", type=str, default="invalid")

args, _ = parser.parse_known_args()

latitude = args.latitude
longitude = args.longitude
sensor_id = args.sensor_id
sensor_name = args.sensor_name
water_level_fg_version = args.water_level_fg_version
weather_fg_version = args.weather_fg_version
fv_version = args.fv_version
model_version = args.model_version

### Login to hopsworks

In [218]:
api_key = os.environ.get("HOPSWORKS_API_KEY")
if not api_key:
    raise RuntimeError("HOPSWORKS_API_KEY is not set")

project = hopsworks.login(
    host="eu-west.cloud.hopsworks.ai",
    port=443,
    project="ml_project",
    api_key_value=api_key,
)
fs = project.get_feature_store()

2026-01-11 18:41:48,888 INFO: Closing external client and cleaning up certificates.
2026-01-11 18:41:48,891 INFO: Connection closed.
2026-01-11 18:41:48,893 INFO: Initializing external client
2026-01-11 18:41:48,894 INFO: Base URL: https://eu-west.cloud.hopsworks.ai:443
2026-01-11 18:41:50,231 INFO: Python Engine initialized.

Logged in to project, explore it here https://eu-west.cloud.hopsworks.ai:443/p/2184


### Extract secrets from the feature store

### Download the model from Model Registry

In [219]:
from utils import get_model_name

mr = project.get_model_registry()

model_name = get_model_name(sensor_name, sensor_id)

retrieved_model = mr.get_model(
    name=model_name,
    version=model_version
)

#Download the saved model artifacts to a local directory
saved_model_dir = retrieved_model.download()

Downloading: 0.000%|          | 0/1333020 elapsed<00:00 remaining<?

Downloading: 0.000%|          | 0/1397527 elapsed<00:00 remaining<?

Downloading: 0.000%|          | 0/1428145 elapsed<00:00 remaining<?

Downloading: 0.000%|          | 0/607210 elapsed<00:00 remaining<?

Downloading: 0.000%|          | 0/1387617 elapsed<00:00 remaining<?

Downloading: 0.000%|          | 0/1403232 elapsed<00:00 remaining<?

Downloading: 0.000%|          | 0/1414005 elapsed<00:00 remaining<?

Downloading: 0.000%|          | 0/1384138 elapsed<00:00 remaining<?

Downloading: 0.000%|          | 0/1383353 elapsed<00:00 remaining<?

In [220]:
retrieved_xgboost_model = XGBRegressor()
retrieved_xgboost_model.load_model(saved_model_dir + f"/{model_name}.json")
retrieved_xgboost_model

XGBRegressor(base_score='2.9236506E4', booster='gbtree', callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None,
             feature_types=['float', 'float', 'float', 'float', 'float',
                            'float', 'float', 'float', 'float', 'float',
                            'float', 'float', 'float', 'float', 'floa...
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

### Get the water level and weather forecast feature view

In [221]:
from utils import get_water_level_features, get_weather_features

water_level_features = get_water_level_features()
weather_features = get_weather_features()

water_level_fg = fs.get_feature_group(
    name=f"water_level_lagged_{sensor_name}_{sensor_id}",
    version=water_level_fg_version,
)

weather_fg = fs.get_feature_group(
    name=f"weather_features_{sensor_name}_{sensor_id}",
    version=weather_fg_version,
)

@hopsworks.udf(
    return_type=[int] * 12,   # month_1 ... month_12
    mode="pandas"
)
def add_month_one_hot(date: pd.Series) -> pd.DataFrame:
    """
    One-hot encode month from event-time column.
    """
    # Extract month (1–12)
    month = pd.to_datetime(date).dt.month

    # One-hot encode
    dummies = pd.get_dummies(month, prefix="month")

    # Ensure ALL 12 months exist (critical!)
    for m in range(1, 13):
        col = f"month_{m}"
        if col not in dummies:
            dummies[col] = 0

    # Stable column order
    dummies = dummies[[f"month_{m}" for m in range(1, 13)]]

    return dummies.astype("int32")

query = (
    water_level_fg
    .select(["date"] + water_level_features)
    .join(
        weather_fg.select(weather_features),
        on=["sensor_id", "date"]
    )
)

fv = fs.get_or_create_feature_view(
    name=f"water_level_training_view_{sensor_name}_{sensor_id}",
    version=fv_version,
    query=query,
    description=(
        "Training feature view for water level prediction. "
        "Combines lagged water level features, weather features, "
        "and on-demand month one-hot encoding."
    ),
    labels=["water_level_cm"],   # prediction target
    transformation_functions=[add_month_one_hot],
)

2026-01-11 18:42:07,631 WARNING: UserWarning: Cannot extract imported dependencies for the UDF from the module in which it is defined. Please make sure to import all dependencies for the UDF inside the function.



### Fetch weather predictions

In [222]:
X_train, y_train = fv.training_data(
    description="bla bla bla"
)
print(X_train.columns)

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (3.65s) from Hopsworks, using Hopsworks Feature Query Service.   Reading data from Hopsworks, using Hopsworks Feature Query Service...   
2026-01-11 18:42:24,009 INFO: Computing insert statistics
2026-01-11 18:42:25,442 WARNING: VersionWarning: Incremented version to `2`.

Index(['date', 'water_level_cm_t_1', 'water_level_cm_t_3',
       'water_level_cm_t_7', 'water_level_cm_t_14', 'precipitation_sum',
       'snowfall_sum', 'rain_sum', 'temperature_2m_mean',
       'wind_speed_10m_mean', 'surface_pressure_mean', 'precip_sum_3d',
       'precip_sum_7d', 'precip_sum_14d', 'snow_sum_14d', 'snow_sum_30d',
       'snow_sum_60d', 'precipitation_sum_n_75km', 'snowfall_sum_n_75km',
       'rain_sum_n_75km', 'temperature_2m_mean_n_75km',
       'wind_speed_10m_mean_n_75km', 'surface_pressure_mean_n_75km',
       'precipitation_sum_s_75km', 'snowfall_sum_s_75km', 'rain_sum_s_75km',
       'temperature_2m_mean_s_75km', 

In [223]:
# ensure datetime dtype
X_train["date"] = pd.to_datetime(X_train["date"])

# sort by time (ascending)
X_train = X_train.sort_values("date").reset_index(drop=True)

# do the same for y_train so rows still match X_train
y_train = y_train.loc[X_train.index].reset_index(drop=True)

# (optional) if y_train has its own date column or index, easiest is to merge instead
print(X_train.tail())

           date  water_level_cm_t_1  water_level_cm_t_3  water_level_cm_t_7  \
2182 2026-01-05             29262.0             29263.0             29269.0   
2183 2026-01-06             29263.0             29262.0             29265.0   
2184 2026-01-07             29263.0             29262.0             29266.0   
2185 2026-01-08             29262.0             29263.0             29266.0   
2186 2026-01-09             29262.0             29263.0             29263.0   

      water_level_cm_t_14  precipitation_sum  snowfall_sum  rain_sum  \
2182              29272.0                0.0          0.00       0.0   
2183              29272.0                0.9          0.63       0.0   
2184              29271.0                2.8          1.96       0.0   
2185              29270.0                6.0          4.20       0.0   
2186              29269.0                0.4          0.28       0.0   

      temperature_2m_mean  wind_speed_10m_mean  ...  add_month_one_hot_date_2  \
2182       

### Get the weather feature group seperately

In [224]:
import time

def get_or_create_feature_view_with_retry(*, retries: int = 1, sleep_s: float = 2.0, **kwargs):
    """
    Call fs.get_or_create_feature_view(**kwargs) with a single retry on any exception.

    Parameters
    ----------
    retries : int
        Number of retries after the first failure (default: 1 -> total 2 attempts).
    sleep_s : float
        Seconds to sleep before retrying.
    **kwargs
        Passed directly to fs.get_or_create_feature_view.

    Returns
    -------
    FeatureView
        The created or retrieved feature view.

    Raises
    ------
    Exception
        Re-raises the last exception if all attempts fail.
    """
    last_exc = None
    for attempt in range(retries + 1):
        try:
            return fs.get_or_create_feature_view(**kwargs)
        except Exception as e:
            last_exc = e
            if attempt >= retries:
                raise
            time.sleep(sleep_s)

    # unreachable, but keeps type checkers happy
    raise last_exc


# 1) build the query for weather only
weather_query = (
    weather_fg
    .select(["sensor_id", "date"] + weather_features)
    # (optional) if you want only one sensor_id inside the view:
    .filter(weather_fg.sensor_id == sensor_id)
)

# 2) create feature view
weather_fv = get_or_create_feature_view_with_retry(
    retries=1,
    sleep_s=2.0,
    name=f"weather_view_{sensor_name}_{sensor_id}",
    version=1,
    query=weather_query,
    description="Weather-only feature view (includes spatial + aggregated weather features).",
    transformation_functions=[add_month_one_hot],
)

In [225]:
df_weather = weather_fv.get_batch_data(data_format="pandas")
print(df_weather)

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (2.68s) from Hopsworks, using Hopsworks Feature Query Service.   
      sensor_id       date  precipitation_sum  snowfall_sum  rain_sum  \
0         20048 2020-02-29                0.4          0.28       0.0   
1         20048 2020-03-01                3.1          2.17       0.0   
2         20048 2020-03-02               12.7          8.89       0.0   
3         20048 2020-03-03                2.9          2.03       0.0   
4         20048 2020-03-04                7.7          5.32       0.1   
...         ...        ...                ...           ...       ...   
2152      20048 2026-01-20                0.5          0.00       0.5   
2153      20048 2026-01-21                0.1          0.00       0.1   
2154      20048 2026-01-22                0.0          0.00       0.0   
2155      20048 2026-01-23                0.0          0.00       0.0   
2156      20048 2026-01-24                0.0         

In [226]:
df_weather["date"] = pd.to_datetime(df_weather["date"])
df_weather = df_weather.sort_values("date").reset_index(drop=True)

print(df_weather.head())
print(df_weather.tail())

   sensor_id       date  precipitation_sum  snowfall_sum  rain_sum  \
0      20048 2020-02-29                0.4          0.28       0.0   
1      20048 2020-03-01                3.1          2.17       0.0   
2      20048 2020-03-02               12.7          8.89       0.0   
3      20048 2020-03-03                2.9          2.03       0.0   
4      20048 2020-03-04                7.7          5.32       0.1   

   temperature_2m_mean  wind_speed_10m_mean  surface_pressure_mean  \
0                 -6.5                  9.1                  960.3   
1                 -9.2                 10.6                  953.1   
2                 -7.6                 15.2                  959.4   
3                 -6.2                 11.6                  976.0   
4                 -4.5                 11.3                  973.6   

   precip_sum_3d  precip_sum_7d  ...  add_month_one_hot_date_2  \
0            0.4            1.1  ...                         0   
1            3.5          

In [227]:
print(df_weather.columns)

Index(['sensor_id', 'date', 'precipitation_sum', 'snowfall_sum', 'rain_sum',
       'temperature_2m_mean', 'wind_speed_10m_mean', 'surface_pressure_mean',
       'precip_sum_3d', 'precip_sum_7d', 'precip_sum_14d', 'snow_sum_14d',
       'snow_sum_30d', 'snow_sum_60d', 'precipitation_sum_n_75km',
       'snowfall_sum_n_75km', 'rain_sum_n_75km', 'temperature_2m_mean_n_75km',
       'wind_speed_10m_mean_n_75km', 'surface_pressure_mean_n_75km',
       'precipitation_sum_s_75km', 'snowfall_sum_s_75km', 'rain_sum_s_75km',
       'temperature_2m_mean_s_75km', 'wind_speed_10m_mean_s_75km',
       'surface_pressure_mean_s_75km', 'precipitation_sum_e_75km',
       'snowfall_sum_e_75km', 'rain_sum_e_75km', 'temperature_2m_mean_e_75km',
       'wind_speed_10m_mean_e_75km', 'surface_pressure_mean_e_75km',
       'precipitation_sum_w_75km', 'snowfall_sum_w_75km', 'rain_sum_w_75km',
       'temperature_2m_mean_w_75km', 'wind_speed_10m_mean_w_75km',
       'surface_pressure_mean_w_75km', 'add_month_on

In [228]:
import pandas as pd
import numpy as np

def get_required_feature_names(model) -> list[str]:
    # Prefer booster feature names (best for XGBRegressor loaded from JSON)
    booster = model.get_booster()
    fn = booster.feature_names
    if fn is None:
        # fallback: scikit-style
        if hasattr(model, "feature_names_in_"):
            fn = list(model.feature_names_in_)
        else:
            raise ValueError("Model has no feature names. Train with pandas DataFrame so names are stored.")
    return list(fn)

print(get_required_feature_names(retrieved_xgboost_model))

['water_level_cm_t_1', 'water_level_cm_t_3', 'water_level_cm_t_7', 'water_level_cm_t_14', 'precipitation_sum', 'snowfall_sum', 'rain_sum', 'temperature_2m_mean', 'wind_speed_10m_mean', 'surface_pressure_mean', 'precip_sum_3d', 'precip_sum_7d', 'precip_sum_14d', 'snow_sum_14d', 'snow_sum_30d', 'snow_sum_60d', 'precipitation_sum_n_75km', 'snowfall_sum_n_75km', 'rain_sum_n_75km', 'temperature_2m_mean_n_75km', 'wind_speed_10m_mean_n_75km', 'surface_pressure_mean_n_75km', 'precipitation_sum_s_75km', 'snowfall_sum_s_75km', 'rain_sum_s_75km', 'temperature_2m_mean_s_75km', 'wind_speed_10m_mean_s_75km', 'surface_pressure_mean_s_75km', 'precipitation_sum_e_75km', 'snowfall_sum_e_75km', 'rain_sum_e_75km', 'temperature_2m_mean_e_75km', 'wind_speed_10m_mean_e_75km', 'surface_pressure_mean_e_75km', 'precipitation_sum_w_75km', 'snowfall_sum_w_75km', 'rain_sum_w_75km', 'temperature_2m_mean_w_75km', 'wind_speed_10m_mean_w_75km', 'surface_pressure_mean_w_75km', 'add_month_one_hot_date_0', 'add_month_one

In [229]:
from datetime import timedelta

def predict_water_level_next_days(
    model,
    X_train: pd.DataFrame,
    y_train: pd.DataFrame,
    df_weather_all: pd.DataFrame,
    horizon_days: int = 14,
    date_col: str = "date",
) -> pd.DataFrame:
    # --- Prepare history series (date -> water_level_cm actual) ---
    Xh = X_train.copy()
    Xh[date_col] = pd.to_datetime(Xh[date_col])

    # y_train might be DataFrame (n,1); get the series
    y_series = y_train.iloc[:, 0].copy()
    hist = pd.DataFrame({date_col: Xh[date_col].values, "water_level_cm": y_series.values})
    hist = hist.sort_values(date_col).reset_index(drop=True)

    # Use a dict for fast lookup of actual/pred values by date
    wl_by_date = dict(zip(hist[date_col], hist["water_level_cm"]))

    last_date = hist[date_col].max()

    # --- Prepare weather dataframe keyed by date ---
    wf = df_weather_all.copy()
    if date_col in wf.columns:
        wf[date_col] = pd.to_datetime(wf[date_col])
        wf = wf.set_index(date_col)
    else:
        # assume index is time-like
        wf.index = pd.to_datetime(wf.index)

    wf = wf.sort_index()

    # --- Determine required feature columns in correct order ---
    required = get_required_feature_names(model)

    preds = []

    for step in range(1, horizon_days + 1):
        d = last_date + timedelta(days=step)

        # Weather features for that day
        if d not in wf.index:
            raise KeyError(f"No weather features available for forecast date {d.date()} in df_weather_all.")

        row = wf.loc[d].to_dict()
        # If wf.loc[d] returns a Series, dict is fine; if it returns multiple rows, error out
        if isinstance(wf.loc[d], pd.DataFrame):
            raise ValueError(f"Multiple weather rows for date {d.date()}; ensure one row per day.")

        # --- Build lag features from wl_by_date (mix of actual & predicted) ---
        def wl_at(date_):
            if date_ not in wl_by_date:
                raise KeyError(f"Missing water level for {date_.date()} needed for lags of {d.date()}.")
            return float(wl_by_date[date_])

        row["water_level_cm_t_1"]  = wl_at(d - timedelta(days=1))
        row["water_level_cm_t_3"]  = wl_at(d - timedelta(days=3))
        row["water_level_cm_t_7"]  = wl_at(d - timedelta(days=7))
        row["water_level_cm_t_14"] = wl_at(d - timedelta(days=14))

        # If your model includes the date column or month-one-hot columns, handle them:
        # - If "date" is part of required features, set it
        if date_col in required:
            row[date_col] = d

        # --- Create one-row DF with EXACT required columns (missing -> 0) ---
        X_row = pd.DataFrame([{col: row.get(col, 0) for col in required}], columns=required)

        # Make sure numeric columns are numeric
        for c in X_row.columns:
            if c != date_col:
                X_row[c] = pd.to_numeric(X_row[c], errors="coerce").fillna(0)

        y_pred = float(model.predict(X_row)[0])

        # store prediction
        preds.append({date_col: d, "water_level_cm_pred": y_pred})

        # feed prediction back for future lags
        wl_by_date[d] = y_pred

    return pd.DataFrame(preds).sort_values(date_col).reset_index(drop=True)


In [230]:
# df_weather_all should contain rows for the forecast horizon dates (daily)
# and columns matching what the model was trained on (weather_features etc.)
pred_df = predict_water_level_next_days(
    retrieved_xgboost_model,
    X_train=X_train,
    y_train=y_train,
    df_weather_all=df_weather,   # <- your forecast features dataframe
    horizon_days=14,
    date_col="date",
)

print(pred_df)

         date  water_level_cm_pred
0  2026-01-10         29247.287109
1  2026-01-11         29245.765625
2  2026-01-12         29243.691406
3  2026-01-13         29241.992188
4  2026-01-14         29239.458984
5  2026-01-15         29237.306641
6  2026-01-16         29234.662109
7  2026-01-17         29231.939453
8  2026-01-18         29229.109375
9  2026-01-19         29226.841797
10 2026-01-20         29225.335938
11 2026-01-21         29221.562500
12 2026-01-22         29219.441406
13 2026-01-23         29217.457031


In [231]:
from pathlib import Path

HERE = Path(__file__).resolve().parent          # notebooks/
REPO_ROOT = HERE.parent                         # repo root (assuming scripts/ is in repo root)

ARTIFACTS_DIR = REPO_ROOT / "artifacts"
PREDICTIONS_DIR = ARTIFACTS_DIR / "predictions"
prediction_file = f"predictions_{sensor_name}_{sensor_id}.csv"

prediction_path = PREDICTIONS_DIR / prediction_file

pred_df.to_csv(prediction_path, index=False)
print(f"Saved predictions to: {prediction_path}")

Saved predictions to: ../artifacts/predictions/predictions_storsjon_jamtland_20048.csv
